<a href="https://colab.research.google.com/github/girinath18/AI_Headshot_RG/blob/Finale/Nokia2API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Right From the this cell**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyngrok
from pyngrok import ngrok
ngrok.set_auth_token("2gbCNOE5wJ9vXAdZQQRDAdCP0EU_47uYqH7uXrjbn7pkTsd7X")
public_url = ngrok.connect(5000).public_url
!git clone https://github.com/girinath18/roop.git
!pip install --upgrade insightface onnxruntime
%cd roop


Cloning into 'roop'...
remote: Enumerating objects: 1540, done.
remote: Total 1540 (delta 0), reused 0 (delta 0), pack-reused 1540
Receiving objects: 100% (1540/1540), 97.43 MiB | 22.75 MiB/s, done.
Resolving deltas: 100% (917/917), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 13.4 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp310-cp310-linux_x86_64.whl size=1055339 sha256=9ab3430be15b951ec86c69bef714acf29671e1cd2c40ad9bc8b27d6a8b639cbc
  Stored in directory: /root/.cache/pip/wheels/e3

In [1]:
!pip install flask_ngrok
!pip install pyngrok
!pip install onnxruntime
!pip install customtkinter
!pip install torch torchvision
!pip install flask
!pip install pyngrok --upgrade
!pip install --upgrade pyngrok
!pip install -r /content/roop/requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
Ignoring tkinterdnd2-universal: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment
Ignoring onnxruntime: markers 'python_version != "3.9" and sys_platform == "darwin" and platform_machine != "arm64"' don't match your environment
Ignoring onnxruntime-coreml: markers 'python_version == "3.9" and sys_platform == "darwin" and platform_machine != "arm64"' don't match your environment
Ignoring onnxruntime-silicon: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment


In [2]:
%rm -rf /root/.insightface/models/buffalo_l/
!pip install --upgrade insightface onnxruntime

# **Trying to encapsulate codeformers function with faceswapAPI**

In [3]:
from IPython.display import clear_output
%cd /content
!rm -rf /content/CodeFormer
!git clone https://github.com/sczhou/CodeFormer.git
%cd /content/CodeFormer
!pip install -r /content/CodeFormer/requirements.txt
# Install basicsr
!python /content/CodeFormer/basicsr/setup.py develop
# Download the pre-trained model
!python /content/CodeFormer/scripts/download_pretrained_models.py facelib
!python /content/CodeFormer/scripts/download_pretrained_models.py CodeFormer
clear_output()

# **Added the GET endpoint to the script**

In [11]:
import logging
import os
import shutil
import subprocess
from flask import Flask, request, jsonify, send_from_directory
from pyngrok import ngrok
import cv2
import matplotlib.pyplot as plt

app = Flask(__name__)

logging.basicConfig(level=logging.INFO)

TEMPLATES_FOLDER_PATH = "/content/Untitled Folder 4/"
OUTPUT_BASE_FOLDER = "/content/output_Bot"
PERMANENT_OUTPUT_FOLDER = "/content/output_Bot/RGR_Outputs"
CODEFORMER_OUTPUT_FOLDER = "/content/output_Bot/CodeFormer_Outputs"
CODEFORMER_FINAL_RESULTS_FOLDER = os.path.join(CODEFORMER_OUTPUT_FOLDER, "final_results")
CODEFORMER_PATH = "/content/CodeFormer"
CODEFORMER_SCRIPT_PATH = "/content/CodeFormer/inference_codeformer.py"
RUN_SCRIPT_PATH = "/content/roop/run.py"

def face_swap(source_image_folder, output_base_folder):
    try:
        output_folder = os.path.join(output_base_folder, "RGR_Outputs")
        os.makedirs(output_folder, exist_ok=True)
        os.makedirs(PERMANENT_OUTPUT_FOLDER, exist_ok=True)
        logging.info(f"Output folder created at: {output_folder}")

        source_images = os.listdir(source_image_folder)
        if not source_images:
            logging.error("No source images found.")
            return
        logging.info(f"Found {len(source_images)} source images.")

        template_images = os.listdir(TEMPLATES_FOLDER_PATH)
        if not template_images:
            logging.error("No template images found.")
            return
        logging.info(f"Found {len(template_images)} template images.")

        source_image_name = source_images[0]
        source_image_path = os.path.join(source_image_folder, source_image_name)

        if os.path.isfile(source_image_path):
            logging.info(f"Processing source image: {source_image_name}")

            processed_image_paths = set()

            for target_image_name in template_images:
                target_image_path = os.path.join(TEMPLATES_FOLDER_PATH, target_image_name)

                if os.path.isfile(target_image_path):
                    logging.info(f"Processing target image: {target_image_name}")

                    output_image_path = os.path.join(output_folder, f"{source_image_name}_{target_image_name}")
                    permanent_output_image_path = os.path.join(PERMANENT_OUTPUT_FOLDER, f"{source_image_name}_{target_image_name}")

                    if output_image_path in processed_image_paths:
                        logging.info(f"Skipping already processed image: {output_image_path}")
                        continue

                    command = (
                        f'python "{RUN_SCRIPT_PATH}" -s "{source_image_path}" -t "{target_image_path}" '
                        f'-o "{output_image_path}" --keep-fps --execution-threads 14 --many-faces '
                        f'--execution-provider cuda --frame-processor face_swapper face_enhancer '
                        f'--output-video-quality 35 --temp-frame-format jpg --max-memory 46'
                    )
                    logging.info(f"Running command: {command}")

                    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                    stdout, stderr = process.communicate()
                    stdout_decoded = stdout.decode('utf-8')
                    stderr_decoded = stderr.decode('utf-8')
                    logging.debug(f"Command stdout: {stdout_decoded}")
                    logging.debug(f"Command stderr: {stderr_decoded}")
                    logging.debug(f"Process return code: {process.returncode}")

                    if process.returncode != 0 or "Segmentation fault" in stderr_decoded:
                        logging.error(f"Failed to process target image: {target_image_name}")
                        logging.error(f"Command output (stderr): {stderr_decoded}")
                    else:
                        logging.info(f"Successfully processed target image: {target_image_name}")
                        logging.info(f"Command output (stdout): {stdout_decoded}")
                        shutil.copy(output_image_path, permanent_output_image_path)
                        processed_image_paths.add(output_image_path)

                    if len(processed_image_paths) >= len(template_images):
                        break

        # After face swap process completes, run the CodeFormer process
        process_codeformer(PERMANENT_OUTPUT_FOLDER, CODEFORMER_OUTPUT_FOLDER)

    except Exception as e:
        logging.exception(f"An error occurred: {e}")

def process_codeformer(input_folder, output_folder):
    try:
        # Clean up previous runs
        if os.path.exists(output_folder):
            shutil.rmtree(output_folder)
        os.mkdir(output_folder)

        # Check if input folder contains any images
        if not os.path.isdir(input_folder) or not os.listdir(input_folder):
            logging.error("The input folder is empty or does not exist. Please provide a valid input folder with images.")
            return

        # Inference the uploaded images
        os.chdir(CODEFORMER_PATH)

        CODEFORMER_FIDELITY = 0.7
        BACKGROUND_ENHANCE = True
        FACE_UPSAMPLE = True

        # Running the inference and saving outputs in specified output folder
        command = f'python "{CODEFORMER_SCRIPT_PATH}" -w {CODEFORMER_FIDELITY} --input_path {input_folder} --output_path {output_folder}'
        if BACKGROUND_ENHANCE:
            command += ' --bg_upsampler realesrgan'
        if FACE_UPSAMPLE:
            command += ' --face_upsample'

        logging.info(f"Running CodeFormer command: {command}")
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()
        stdout_decoded = stdout.decode('utf-8')
        stderr_decoded = stderr.decode('utf-8')
        logging.debug(f"Command stdout: {stdout_decoded}")
        logging.debug(f"Command stderr: {stderr_decoded}")
        logging.debug(f"Process return code: {process.returncode}")

        if process.returncode != 0:
            logging.error(f"Command failed with return code: {process.returncode}")
            logging.error(f"Command output (stderr): {stderr_decoded}")
            return

        # Check if output folder contains any images
        output_files = os.listdir(output_folder)
        if not output_files:
            logging.error("No output images found. Something went wrong.")
            return

        # Display Results
        input_image_path = os.path.join(input_folder, os.listdir(input_folder)[0])
        output_image_path = os.path.join(output_folder, output_files[0])

        input_image = imread(input_image_path)
        output_image = imread(output_image_path)
        display(input_image, output_image)

    except Exception as e:
        logging.exception(f"An error occurred in CodeFormer process: {e}")

def display(img1, img2):
    fig = plt.figure(figsize=(25, 10))
    ax1 = fig.add_subplot(1, 2, 1)
    plt.title('Input', fontsize=16)
    ax1.axis('off')
    ax2 = fig.add_subplot(1, 2, 2)
    plt.title('CodeFormer', fontsize=16)
    ax2.axis('off')
    ax1.imshow(img1)
    ax2.imshow(img2)

def imread(img_path):
    img = cv2.imread(img_path)
    if img is None:
        raise FileNotFoundError(f"Image file not found at {img_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

@app.route('/faceswap', methods=['POST'])
def faceswap_api():
    try:
        source_image_folder = "/content/source_images"
        os.makedirs(source_image_folder, exist_ok=True)

        source_images = request.files.getlist('source_images')
        if not source_images:
            logging.error("No images uploaded.")
            return jsonify({"error": "No images uploaded"}), 400

        for source_image in source_images:
            source_image.save(os.path.join(source_image_folder, source_image.filename))
            logging.info(f"Uploaded image: {source_image.filename}")

        face_swap(source_image_folder, OUTPUT_BASE_FOLDER)

        shutil.rmtree(source_image_folder)

        return jsonify({"message": "Face swap and CodeFormer processes initiated."}), 200

    except Exception as e:
        logging.exception(f"An error occurred: {e}")
        return jsonify({"error": "An error occurred"}), 500

@app.route('/get_images', methods=['GET'])
def get_images():
    try:
        output_folder = CODEFORMER_FINAL_RESULTS_FOLDER
        logging.info(f"Looking for images in folder: {output_folder}")

        if not os.path.exists(output_folder):
            logging.error(f"Output folder {output_folder} does not exist")
            return jsonify({"error": "Output folder does not exist"}), 404

        image_files = [f for f in os.listdir(output_folder) if os.path.isfile(os.path.join(output_folder, f))]
        logging.info(f"Found image files: {image_files}")

        if not image_files:
            logging.info("No image files found in the output folder")
            return jsonify({"images": []}), 200

        image_links = [f"{request.host_url}get_image/{image}" for image in image_files]
        logging.info(f"Generated image links: {image_links}")

        return jsonify({"images": image_links}), 200

    except Exception as e:
        logging.exception(f"An error occurred while fetching images: {e}")
        return jsonify({"error": "An error occurred"}), 500

@app.route('/get_image/<filename>', methods=['GET'])
def get_image(filename):
    try:
        logging.info(f"Serving image: {filename}")
        return send_from_directory(CODEFORMER_FINAL_RESULTS_FOLDER, filename, as_attachment=False)

    except Exception as e:
        logging.exception(f"An error occurred while fetching the image: {e}")
        return jsonify({"error": "An error occurred"}), 500

if __name__ == '__main__':
    url = ngrok.connect(5000)
    print(f" * Ngrok tunnel URL: {url}")

    app.run(port=5000)


 * Ngrok tunnel URL: NgrokTunnel: "https://5f4b-34-34-127-192.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:root:Failed to process target image: OLD (1).jpeg
ERROR:root:Command output (stderr): Segmentation fault (core dumped)

ERROR:root:Failed to process target image: OLD (6).jpeg
ERROR:root:Command output (stderr): Segmentation fault (core dumped)

ERROR:root:Failed to process target image: OLD (2).jpeg
ERROR:root:Command output (stderr): Segmentation fault (core dumped)

ERROR:root:Failed to process target image: OLD (7).jpeg
ERROR:root:Command output (stderr): Segmentation fault (core dumped)

ERROR:root:Failed to process target image: OLD (8).jpeg
ERROR:root:Command output (stderr): Segmentation fault (core dumped)

ERROR:root:Failed to process target image: OLD (5).jpeg
ERROR:root:Command output (stderr): Segmentation fault (core dumped)

ERROR:root:Failed to process target imag